In [1]:
'''
训练深度神经网络是困难的。使它们在合理的时间内收敛可能是棘手的。在本节中，我们将介绍批量归一化（Batch Normalization），这是一种流行且有效的技术，能够始终加速深度网络的收敛（Ioffe和Szegedy，2015）。批量归一化与残差块（稍后在第8.6节中介绍）一起，使实践者能够常规地训练具有100多层的网络。批量归一化的一个次要（意外的）好处在于其固有的正则化作用。

批量归一化的核心思想是在训练过程中对每个小批量（mini-batch）的数据进行归一化，使其具有零均值和单位方差。这样做可以减少内部协变量偏移（Internal Covariate Shift），即网络层之间输入数据分布的变化。由于梯度下降优化过程依赖于每一层的输入数据分布，减少内部协变量偏移可以使网络更快地收敛。

批量归一化通常应用在每个卷积层或全连接层之后，激活函数之前。对于每个小批量数据，批量归一化层计算其均值和方差，并使用这些统计量对数据进行归一化。此外，批量归一化层还引入了两个可学习参数，即缩放因子（scale factor）和平移因子（shift factor），用于恢复数据的原始范围和均值。这些可学习参数使得网络可以自适应地调整归一化操作，以获得最佳性能。

批量归一化的一个副作用是它具有一定的正则化效果。由于每个小批量数据的统计量在一定程度上受到随机性的影响，归一化操作会引入一定的噪声。这种噪声可以防止网络过拟合，从而起到正则化的作用。然而，这种正则化效果通常不足以完全替代其他正则化技术，如权重衰减和dropout。

总之，批量归一化是一种有效的技术，可以加速深度神经网络的收敛，并提供一定的正则化效果。它已经成为现代神经网络架构中的标准组件，并与其他技术（如残差块）结合使用，使得训练超过100层的深度网络成为可能。
'''
import torch
from torch import nn
from d2l import torch as d2l

'''8_1_deep_convolutional_nerual_networks_alexnet.ipynb
在处理数据时，我们通常会在训练之前进行预处理。关于数据预处理的选择通常对最终结果产生巨大影响。回顾我们在第5.7节中将MLP应用于预测房价的例子。我们在处理实际数据时的第一步是将输入特征标准化，使其具有零均值和单位方差（Friedman，1987），经常将后者重新缩放，使得对角线为1，即。另一种策略是将向量重新缩放为单位长度，可能是每个观测值的零均值。这种方法在某些情况下可以工作得很好，例如用于空间传感器数据。这些预处理技术以及许多其他技术有助于保持估计问题的良好控制。关于特征选择和提取的综述，可以参考Guyon等人（2008）的文章。

标准化向量还有一个很好的副作用，即限制作用于其上的函数的复杂性。例如，支持向量机中著名的半径-边缘界限（Vapnik，1995）和感知器收敛定理（Novikoff，1962）依赖于有界范数的输入。

总之，在处理数据时，预处理是一个重要的步骤，它可以帮助我们更好地控制估计问题，提高模型的性能。通过选择合适的预处理方法，如特征标准化、重新缩放或向量标准化，我们可以使模型更容易收敛，提高训练效率，并限制函数的复杂性。在实际应用中，我们需要根据数据的特点和任务需求来选择合适的预处理方法。
'''

In [4]:
def batch_norm(X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # Use is_grad_enabled to determine whether we are in training mode
    if not torch.is_grad_enabled():
        # In prediction mode, use mean and variance obtained by moving average
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # When using a fully connected layer, calculate the mean and
            # variance on the feature dimension
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # When using a two-dimensional convolutional layer, calculate the
            # mean and variance on the channel dimension (axis=1). Here we
            # need to maintain the shape of X, so that the broadcasting
            # operation can be carried out later
            mean = X.mean(dim=(0, 2, 3), keepdim=True)
            var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True)
        # In training mode, the current mean and variance are used
        X_hat = (X - mean) / torch.sqrt(var + eps)
        # Update the mean and variance using moving average
        moving_mean = (1.0 - momentum) * moving_mean + momentum * mean
        moving_var = (1.0 - momentum) * moving_var + momentum * var
    Y = gamma * X_hat + beta  # Scale and shift
    return Y, moving_mean.data, moving_var.data

In [5]:
class BatchNorm(nn.Module):
    # num_features: the number of outputs for a fully connected layer or the
    # number of output channels for a convolutional layer. num_dims: 2 for a
    # fully connected layer and 4 for a convolutional layer
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # The scale parameter and the shift parameter (model parameters) are
        # initialized to 1 and 0, respectively
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # The variables that are not model parameters are initialized to 0 and
        # 1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def forward(self, X):
        # If X is not on the main memory, copy moving_mean and moving_var to
        # the device where X is located
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
        # Save the updated moving_mean and moving_var
        Y, self.moving_mean, self.moving_var = batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.1)
        return Y

In [6]:
class BNLeNetScratch(d2l.Classifier):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5), BatchNorm(6, num_dims=4),
            nn.Sigmoid(), nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(16, kernel_size=5), BatchNorm(16, num_dims=4),
            nn.Sigmoid(), nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(), nn.LazyLinear(120),
            BatchNorm(120, num_dims=2), nn.Sigmoid(), nn.LazyLinear(84),
            BatchNorm(84, num_dims=2), nn.Sigmoid(),
            nn.LazyLinear(num_classes))

In [7]:
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128)
model = BNLeNetScratch(lr=0.1)
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [2]:
'''8_1_deep_convolutional_nerual_networks_alexnet.ipynb
直观地说，这种标准化与我们的优化器相互配合，因为它先验地将参数放在相似的尺度上。因此，很自然地会问，在深度网络内部是否有一个相应的归一化步骤可能是有益的。虽然这并不完全是导致批量归一化（Ioffe和Szegedy，2015）发明的原因，但这是一种理解批量归一化及其表亲层归一化（Ba等人，2016）在统一框架内的有用方法。

其次，对于典型的MLP或CNN，在训练过程中，中间层的变量（例如，MLP中的仿射变换输出）可能具有不同数量级的值：沿着从输入到输出的层、同一层中的单元以及由于我们对模型参数的更新而随时间变化。批量归一化的发明者非正式地假设，这种变量分布的漂移可能会阻碍网络的收敛。直观地说，我们可能会猜测，如果一层的可变激活是另一层的100倍，这可能需要在学习率中进行补偿性调整。自适应求解器，如AdaGrad（Duchi等人，2011）、Adam（Kingma和Ba，2014）、Yogi（Zaheer等人，2018）或分布式Shampoo（Anil等人，2020）旨在从优化的角度解决这个问题，例如，通过添加二阶方法的方面。另一种方法是通过自适应归一化来防止问题的发生。

第三，更深层次的网络更复杂，更容易过拟合。这意味着正则化变得更加重要。一种常见的正则化技术是噪声注入。这种方法已经被认识很长时间了，例如，关于输入的噪声注入（Bishop，1995）。它还构成了第5.6节中dropout的基础。事实证明，相当意外地，批量归一化传达了所有三个优点：预处理、数值稳定性和正则化。

批量归一化应用于单个层，或者可选地应用于所有层：在每次训练迭代中，我们首先通过减去均值并除以标准差来对输入（批量归一化）进行归一化，其中这两者都是基于当前小批量的统计数据估计的。接下来，我们应用一个缩放系数和一个偏移量来恢复丢失的自由度。正是由于这种基于批量统计数据的归一化，批量归一化得名。

请注意，如果我们尝试使用大小为1的小批量应用批量归一化，我们将无法学到任何东西。这是因为在减去均值之后，每个隐藏单元的值将为0。如您所猜，由于我们将整个部分用于批量归一化，对于足够大的小批量，该方法被证明是有效且稳定的。这里的一个启示是，在应用批量归一化时，批量大小的选择比没有批量归一化更重要，或者至少，在我们可能调整批量大小时需要适当的校准。
''' 

''' 
用表示一个小批量，令是批量归一化的输入（）。在这种情况下，批量归一化定义如下：

(8.5.1)
 

在(8.5.1)中， 是小批量 的样本均值， 是小批量 的样本标准差。应用标准化后，生成的小批量具有零均值和单位方差。选择单位方差（而不是其他一些神奇的数字）是任意的。我们通过包括与 相同形状的逐元素比例参数 和偏移参数 来恢复这个自由度。这两个参数都是需要作为模型训练的一部分进行学习的参数。

由于批量归一化会积极地将中间层的变量中心化和重新缩放回给定的均值和大小（通过 和 ），所以在训练过程中中间层的变量幅度不能发散。实际经验证实，正如在讨论特征重新缩放时所暗示的，批量归一化似乎允许更激进的学习率。我们根据以下公式计算(8.5.1)中的 和 ：

(8.5.2)
 
 
 
 
注意，我们在方差估计中添加了一个很小的常数 ，以确保我们永远不会尝试除以零，即使在经验方差估计可能非常小或消失的情况下。估计值 和 通过使用均值和方差的嘈杂估计来抵消缩放问题。你可能会认为这种噪声应该是一个问题。相反，它实际上是有益的。

这在深度学习中成为一个反复出现的主题。出于尚未在理论上得到很好描述的原因，优化中的各种噪声来源通常导致更快的训练和更少的过拟合：这种变化似乎起到了一种正则化的作用。Teye等人（2018）和Luo等人（2018）分别将批量归一化的属性与贝叶斯先验和惩罚联系起来。特别地，这为解释为什么批量归一化在50-100范围内的适中小批量大小上效果最好提供了一些线索。这种特定大小的小批量似乎在每层注入了“适量”的噪声，无论是通过 来调整尺度，还是通过 来调整偏移：较大的小批量由于更稳定的估计而正则化较少，而微小的小批量由于高方差而破坏有用的信号。进一步探索这个方向，考虑其他类型的预处理和过滤可能导致其他有效类型的正则化。

修复一个训练好的模型，你可能会认为我们更愿意使用整个数据集来估计均值和方差。一旦训练完成，为什么我们会希望同一张图片被分类为不同的类别，这取决于它所在的批次？在训练过程中，这种精确计算是不可行的，因为所有数据示例的中间变量在我们更新模型时会发生变化。然而，一旦模型训练完成，我们可以根据整个数据集计算每个层变量的均值和方差。实际上，这是采用批量归一化的
模型的标准做法；因此，在训练模式下（通过小批量统计数据进行归一化）和预测模式下（通过数据集统计数据进行归一化），批量归一化层的功能有所不同。在这种形式下，它们与第5.6节中的dropout正则化的行为密切相关，其中噪声仅在训练过程中注入。

综上所述，批量归一化在深度学习中具有重要作用，它帮助解决预处理、数值稳定性和正则化问题。在训练和预测过程中，批量归一化层的功能有所不同，这使得模型在训练过程中能够适应不同的数据分布，并在预测过程中提供更准确的结果。此外，在应用批量归一化时，批量大小的选择变得更加重要，或者至少需要在调整批量大小时进行适当的校准。
''' 


class BNLeNet(d2l.Classifier):
    def __init__(self, lr=0.1, num_classes=10):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nn.LazyConv2d(6, kernel_size=5), nn.LazyBatchNorm2d(),
            nn.Sigmoid(), nn.AvgPool2d(kernel_size=2, stride=2),
            nn.LazyConv2d(16, kernel_size=5), nn.LazyBatchNorm2d(),
            nn.Sigmoid(), nn.AvgPool2d(kernel_size=2, stride=2),
            nn.Flatten(), nn.LazyLinear(120), nn.LazyBatchNorm1d(),
            nn.Sigmoid(), nn.LazyLinear(84), nn.LazyBatchNorm1d(),
            nn.Sigmoid(), nn.LazyLinear(num_classes))

In [3]:
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128)
model = BNLeNet(lr=0.1)
model.apply_init([next(iter(data.get_dataloader(True)))[0]], d2l.init_cnn)
trainer.fit(model, data)

/data2/miniconda3/envs/d2l-pytorch/lib/python3.8/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
